# Кластеризация методом HDBSCAN
Используем реализацию [HDBSCAN](https://scikit-learn.org/stable/modules/clustering.html#hdbscan) в [scikit-learn](https://scikit-learn.org/stable/index.html) (класс [HDBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html#sklearn.cluster.HDBSCAN))

In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import HDBSCAN
from sklearn.preprocessing import StandardScaler  # z-нормировка
# from sklearn.preprocessing import MinMaxScaler, RobustScaler, MaxAbsScaler # другие способы нормировки
from sklearn.manifold import TSNE # Метод t-SNE для визуализации

import seaborn as sns # 2D-визуализация
import plotly.express as px # 3D-визуализация

import matplotlib.pyplot as plt

# Не показывать Warning
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
df = pd.read_csv('./datasets/Countries.csv')
# Удалим нечисловые переменные
X = df.drop(columns=['Страны'])
X.index = df['Страны']
X

## Нормирование данных

In [ ]:
# Специфицируем метод нормировки
scaler = StandardScaler(with_mean=True, with_std=True)
# scaler = MinMaxScaler()
# scaler = RobustScaler()
# scaler = MaxAbsScaler()

X_norm = scaler.fit_transform(X)

## Кластеризация

*Важно*: выбросы имеют номер -1!

In [ ]:
# Специфицируем процедуру
cluster = HDBSCAN(min_cluster_size=5, min_samples=None)
# проводим кластеризацию
cluster.fit(X_norm)

print(cluster.labels_)

In [ ]:
n_clusters_ = len(set(cluster.labels_)) - (1 if -1 in cluster.labels_ else 0)
n_noise_ = list(cluster.labels_).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

## Визуализация
используем библиотеки [seaborn](https://seaborn.pydata.org/index.html) и [plotly](https://plotly.com/python/)

### Визуализация в исходных переменных

In [ ]:
X['Cluster'] = cluster.labels_.astype(str)
X.head()

2D визуализация в seaborn

In [ ]:
# Визуализация в seaborn
# sns.scatterplot(data=X, x='ИРЧП', y='Население', hue='Cluster')
# sns.scatterplot(data=X, x='ИРЧП', y='ВВП д/н', hue='Cluster')
# plt.show()

2D визуализация в plotly

In [ ]:
fig = px.scatter(X, x='ИРЧП', y='Население', color='Cluster', hover_name=X.index)
# fig.update_traces(marker_size=6)
fig.show()

In [ ]:
fig = px.scatter(X, x='ИРЧП', y='ВВП д/н', color='Cluster', hover_name=X.index)
# fig.update_traces(marker_size=5)
fig.show()

3D визуализация в plotly

In [ ]:
fig = px.scatter_3d(X, x='ИРЧП', y='ВВП д/н', z='Безработица (%)', color='Cluster', hover_name=X.index)
fig.update_traces(marker_size=4)
fig.show()

### Визуализация после снижения размерности

In [ ]:
tsne = TSNE(n_components=2, perplexity=10, random_state=2)
X_embedded = tsne.fit_transform(X_norm)

In [ ]:
sns.scatterplot(x=X_embedded[:,0], y=X_embedded[:,1], hue=cluster.labels_.astype(str))
plt.show()